In [23]:
import pandas as pd

analysis = 'CPO_CUSTOM'
df = pd.read_csv(f'/root/Logical-Fallacies/src/EVAL/AMT/annotations/SFT_vs_{analysis}.csv')
df = df [['Input.topic', 'Input.stance', 'Input.argument1', 'Input.argument2', 'Input.sft_index', 'Answer.q1_valid']]


In [24]:
set_topic = (df['Input.topic'].unique())
seen_topics = {topic: [] for topic in set_topic}
for i, entry in df.iterrows():
    seen_topics[entry['Input.topic']].append(entry['Answer.q1_valid'])

In [25]:
from collections import Counter
is_fallacy = []
preds = []


results = {'SFT': 0, analysis: 0, 'tie_good': 0, 'tie_bad': 0}
l, l2=0, 0
K=0
good, bad = 0,0
A,B=0,0
L = 0
comparisons = []

T , X = 0,0
total = 0
for k,v in seen_topics.items():
    
    sft_index = df[df['Input.topic'] == k].iloc[0]['Input.sft_index']
    counts = Counter(v)

   
    if not all([count == 1 for count in counts.values()]):
        maximum_frequency = max(counts, key=counts.get)
        if maximum_frequency == -10:
            results['tie_bad'] += 1
            winner = 'tie'
        elif maximum_frequency == 10:
            results['tie_good'] += 1
            winner = 'tie'
        elif maximum_frequency in [1,2]:
            maximum_frequency -= 1
            if maximum_frequency != sft_index:
                results[analysis] += 1
                winner = analysis
            else:
                results['SFT'] += 1
                winner = 'SFT'

        
        

In [26]:
results

{'SFT': 34, 'CPO_CUSTOM': 67, 'tie_good': 36, 'tie_bad': 9}

In [27]:
results

{'SFT': 34, 'CPO_CUSTOM': 67, 'tie_good': 36, 'tie_bad': 9}

In [303]:
l/results[analysis], A/results['SFT'], L/len(seen_topics), K/len(seen_topics)

(0.6666666666666666,
 0.7166666666666667,
 0.0855614973262032,
 0.058823529411764705)

For DPO. 79.5% of the time they select DPO as a winner, it is not a fallacy. So only 20% of the time they select DPO as a fallacy

Basically, we should detect when the humans have chosen a sample, if they select a fallacy statement. So basically, when they pick DPO for example, does gpt4 classify it as a fallacy? if yes, then it's bad, otherwise it's good and DPO truly is better. 
When they select SFT, do they select a fallacy statement?

In [304]:
print(l/len(seen_topics))
print(A/len(seen_topics))


0.26737967914438504
0.22994652406417113


Basically, we should penalize if it chose a fallacy statement.

In [305]:
results

{'SFT': 60, 'CPO': 75, 'tie_good': 7, 'tie_bad': 7}

In [306]:
d = {'SFT': results['SFT'], analysis: results[analysis], 'tie': results['tie_good'] + results['tie_bad']}

In [307]:
total

0

In [308]:
total = sum(d.values())
for k,v in d.items():
    print(f'{k}: {v} ({v/total*100:.2f}%)')

SFT: 60 (40.27%)
CPO: 75 (50.34%)
tie: 14 (9.40%)


In [3]:
import pandas as pd
pd.read_csv('/root/Logical-Fallacies/src/EVAL/AMT/rewardsaccuracies.csv')['misunderstood-firebrand-238 - train/rewards/accuracies']


0      0.03125
1      0.04375
2      0.06250
3      0.14375
4      0.16250
        ...   
138    1.00000
139    1.00000
140    1.00000
141    1.00000
142    1.00000
Name: misunderstood-firebrand-238 - train/rewards/accuracies, Length: 143, dtype: float64